# Aplicación del Filtro Savitsky Golay para los productos NDVI del MOD13Q1

## Importando librerías

In [1]:
import numpy as np ## Para realizar cálculos con matrices 
import pandas as pd ## Para la manipulación de datos en un dataframe
import ee # # Para autenticarse e iniciar en Earth Engine
import geemap # Un módulo dinámico de Google Earth Engine
import fiona # Una API de GDAL en Python
from shapely.geometry import shape ## Manipulación de geometrías 
import os #Para asignar y modificar directorios
import json # Para crear objetos en formato .json
from scipy import signal ## Para la aplicación del filtro Savitsky Golay
import rasterio as rio ## Para la amnipulación de arhivos raster
from rasterio.enums import Resampling ## Para hacer el resampleo de los raster

## Inicializando en Google Earth Engine

In [2]:
## Función para inciializar en Google Earth Engine
def ee_initialize(): 
    try:
        ee.Initialize()
    except:
        ee.Authenticate()
        ee.Initialize()

In [3]:
ee_initialize() ## Inicializando Google Earth Engine

## Definiendo la región de interés de la zona

In [4]:
## Definiendo el directorio de trabajo principal
workspace = r"/home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR" ## **Modificar el directorio** ##

In [5]:
def roi(filepath):
    with fiona.open(filepath, "r", enabled_drivers="GPKG") as source:
        for f in source:
            geom = shape(f['geometry']).bounds            
           ## Generando coordenadas a partir de los bounds
            p1   = [geom[0], geom[3]] ## minx y maxy
            p2  = [geom[0], geom[1]] ## minx y miny
            p3  = [geom[2], geom[1]] ## maxx y miny 
            p4  = [geom[2], geom[3]] ## maxx y maxy
           
        return [p1, p2, p3, p4]

In [6]:
shape_path = os.path.join(workspace,"Materiales/Cutervo.gpkg")
## Obteniendo la ROI del  feature
roi_cutervo = roi(shape_path)

##  Obteniendo la coleccion de imagenes MODIS13Q1 para el NDVI y banda de calidad (QA)

In [7]:
## Obteniendo el nombre de las bandas del MOD13Q1
ee.ImageCollection("MODIS/006/MOD13Q1").aggregate_array('system:band_names').getInfo()[0]

['NDVI',
 'EVI',
 'DetailedQA',
 'sur_refl_b01',
 'sur_refl_b02',
 'sur_refl_b03',
 'sur_refl_b07',
 'ViewZenith',
 'SolarZenith',
 'RelativeAzimuth',
 'DayOfYear',
 'SummaryQA']

In [8]:
aoi = ee.Geometry.Polygon(roi_cutervo, None, False)
# Factor de escala para el producto MODIS MOD13Q1 
def scale_factor(image):
    return image.multiply(0.0001).copyProperties(image, ['system:time_start','system:time_end']) ## Copiando propiedades de la fecha de cada imagen

In [9]:
## Filtrando las imagenes por Google Earth Engine
## Colección de imagenes del NDVI
ndvi_dataset= ee.ImageCollection("MODIS/006/MOD13Q1").select('NDVI')\
                                 .filter(ee.Filter.date('2001-01-01', '2018-12-31'))\
                                 .map(scale_factor)
## Colección de imagenes de la QA
qa_dataset = ee.ImageCollection("MODIS/006/MOD13Q1").select('DetailedQA')\
                                .filter(ee.Filter.date('2001-01-01', '2018-12-31'))
count = int(ndvi_dataset.size().getInfo()) ## Número de imagenes del Image Collection
ndvi_images = ndvi_dataset.toList(count)  ## Lista de las imagenes NDVI
qa_images = qa_dataset.toList(count)  ## Lista de las imagenes de QA

## Filtro de calidad por omisión de los píxeles malos

### Calculando los píxeles malos en binario

In [10]:
## Obteniendo los binarios  a partir de un decimal
def binarizar(decimal):
    binario = ''
    while decimal // 2 != 0:
        binario = str(decimal % 2) + binario
        decimal = decimal // 2
    return "0"*(16-(len(binario)+1))+ str(decimal) + binario

In [11]:
## Binarios en el rango de los decimales de 1 a 65535.
bin_data =[]
for i in list(range(1,65536)):
           bin_data.append(binarizar(i))

In [12]:
## Creando un dataframe con las columnas binario y decimal
d = {'bin': bin_data, 'decimal': list(range(1,65536))}
data_frame = pd.DataFrame(data=d)
data_frame

,bin,decimal
0,0000000000000001,1
1,0000000000000010,2
2,0000000000000011,3
3,0000000000000100,4
4,0000000000000101,5
...,...,...
65530,1111111111111011,65531
65531,1111111111111100,65532
65532,1111111111111101,65533
65533,1111111111111110,65534


In [13]:
## Filtrando los binarios que indican pixeles malos.
data_bindf = data_frame[data_frame['bin'].apply(lambda x:
                                                x[0] == "1" or x[5] == "1" or x[7] == "1"  ## Yes on mixed clouds, possible shadow and adjacent cloud detected 
                                                or x[8:10] == "00" or x[8:10] == "11" or x[10:14] == "1111" ## Aerosol quantity high and climatology, Not useful for any other reason/not processed
                                                or x[10:14] == "1110" or x[10:14] == "1101" or x[10:14] == "1100" ## L1B data faulty, Quality so low that it is not useful and Lowest quality 
                                                or x[14:16] == "10" or x[14:16] == "11")] ## Pixel produced, but most probably cloud  and not produced due to other reasons than clouds
data_bindf

,bin,decimal
0,0000000000000001,1
1,0000000000000010,2
2,0000000000000011,3
3,0000000000000100,4
4,0000000000000101,5
...,...,...
65530,1111111111111011,65531
65531,1111111111111100,65532
65532,1111111111111101,65533
65533,1111111111111110,65534


In [14]:
## Definiendo antes la transformación afín de cada raster según los bounds del layer
def transform(filepath):
    with fiona.open(filepath, "r", enabled_drivers="GPKG") as source:
        for f in source:
            geom = shape(f['geometry']).bounds
        return geom
    
## Luego, definimos una función para escribir los matrices resultantes en raster
def writeRaster(arr,filepath, out_dir):
        new_dataset = rio.open(out_dir, 'w', driver='GTiff',
                                    height = arr.shape[0], width = arr.shape[1],
                                    count=1, dtype=str(arr.dtype),
                                    crs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs ',
                                    transform=rio.transform.from_bounds(transform(filepath)[0], transform(filepath)[1],transform(filepath)[2], transform(filepath)[3],
                                                                                                                   arr.shape[1], arr.shape[0]))
        new_dataset.write(arr, 1)
        new_dataset.close()

### Obteniendo los raster con el filtro de calidad

In [15]:
for i in range(0, count):
        ## Obteniendo array a partir del Google Earth Engine
        ndvi_array=np.array(ee.Image(ndvi_images.get(i))\
                                               .sampleRectangle(region=aoi, defaultValue=-2)\
                                               .get("NDVI").getInfo())
        qa_array = np.array(ee.Image(qa_images.get(i))\
                                              .sampleRectangle(region=aoi, defaultValue=0)\
                                              .get("DetailedQA").getInfo())
        
        ##  Cambiar los valores por defecto a NaN values
        ndvi_arr= np.where(ndvi_array==-2,np.nan,ndvi_array)
        qa_arr = np.where(ndvi_array== 0,np.nan,qa_array)
        
        ##  Filtrando el NDVI sin contar los pixeles malos
        mask = np.isin(qa_arr, np.array(data_bindf.decimal))
        qa_arr[mask] = np.nan
        qa_arr[~mask] = 1
        ndvi_filter = ndvi_arr*qa_arr
        
        ## Guardandolo como raster
        start = ee.Date(ee.Image(ndvi_images.get(i)).get("system:time_start")).format('YYYY-MM-dd').getInfo() 
        end = ee.Date(  ee.Image(ndvi_images.get(i)).get("system:time_end")).format('YYYY-MM-dd').getInfo()
        filename= "MOD13Q1_NDVI_filter_250m_16days" + "_" + "{}".format(start) +"_" +"{}".format(end) + ".tif"
        filtrado = os.path.join( workspace,"RasterData/MODQ1/Filtrado")
        out_dir = os.path.join(filtrado,filename)
        writeRaster(ndvi_filter,shape_path,out_dir)

## Resampleando los raster de 250m a 5km

In [16]:
def rescaling(tif_path,out_dir, scale):    
        with rio.open(tif_path) as dataset:
            
            # resample data to target shape
            data =dataset.read( 
                out_shape=(
                    dataset.count,
                    int(dataset.height * scale),
                    int(dataset.width * scale)
                ),
                resampling=Resampling.bilinear
            )

            # scale image transform
            transform = dataset.transform * dataset.transform.scale(
                (dataset.width / data.shape[-1]),
                (dataset.height / data.shape[-2])
            )
          
        new_dataset = rio.open(out_dir, 'w', driver='GTiff',
                                    height = int(dataset.height * scale) , width =  int(dataset.width * scale),
                                    count=1, dtype=str(data.dtype),
                                    crs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs ',
                                    transform = dataset.transform * dataset.transform.scale(
                                    (dataset.width / data.shape[-1]),
                                    (dataset.height / data.shape[-2])))
        new_dataset.write(data)
        new_dataset.close()

In [17]:
filtrado = os.path.join(workspace,"RasterData/MODQ1/Filtrado")
resampleado = os.path.join(workspace,"RasterData/MODQ1/Resampleado")
for t in os.listdir(filtrado):
        filename= "MOD13Q1_NDVI_resamp_5km_16days" + "_" + "{}".format(t.split("_")[5]) +"_" +"{}".format(t.split("_")[6])
        out_dir = os.path.join(resampleado,filename)
        rescaling(os.path.join(filtrado, t),out_dir, scale=1/20)

## Aplicando el filtro Savistky Golay

In [18]:
def fixed_index(index):
    fixed = []
    for a in range(0,len(index)):
        mean = np.nanmean(index,axis=0)
        fixed.append(np.where(np.isnan(index[a]),mean,index[a]))
    return fixed

In [19]:
index=[]
date = []
for t in os.listdir(resampleado):
        with rio.open(os.path.join(resampleado,t)) as src:
                    index.append(src.read(1))        
                    transform_res = src.transform
                    date.append(t.split("_")[5])
ind_ = fixed_index(index)

# Dictionary
data = {"Date":date, "Index": ind_}

# Data frame
df = pd.DataFrame(data,columns = ["Date","Index"])

# Converting to date time
df["Date"] =pd.to_datetime(df["Date"], format='%Y-%m-%d', errors='coerce')
df.sort_values(by="Date", key=pd.to_datetime, inplace=True)

##  Filepath for SavGol
savgol = os.path.join(workspace,"RasterData/MODQ1/SavGol")

## Agrupar por mes
df['month'] = pd.DatetimeIndex(df['Date']).month

##Aplicando el filtro de Savitsky Golay
## Agrupando matrices para faciltar el cálculo
stack=np.stack(pd.Series(df["Index"]))

## Agrupando para que evaluar cada pixel a través del tiempo
stack_tp =np.transpose(stack)

## Aplicando el filtro de Savistky Golay
savgol_filter = signal.savgol_filter(stack_tp,7,4)

## Volviendolo a su forma original
savgol_origin =np.transpose(savgol_filter)

In [21]:
res_list=[]
for i in os.listdir(resampleado):
       res_list.append(os.path.join(resampleado,i))
# Read metadata of first file
with rio.open(res_list[0]) as src0:
            meta = src0.meta
            transform_res = src0.transform
# Update meta to reflect the number of layers
meta.update(count = len(res_list))            

In [22]:
## Crando raster por cada matriz 
for i in range(0,len(df["Index"])):
    filename= "MOD13Q1_NDVI_SavGol_5km_16days" + "_" + "{}".format(str(df["Date"][i]).split(" ")[0]) + ".tif"
    out_dir = os.path.join(savgol,filename)
    arr=savgol_origin[i]
    new_dataset = rio.open(out_dir, 'w', driver='GTiff',
                            height = arr.shape[0], width = arr.shape[1],
                            count=1, dtype=str(arr.dtype),
                            crs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs ',
                            transform= transform_res)
    new_dataset.write(arr, 1)
    new_dataset.close()

In [23]:
tif_list=[]
for i in os.listdir(savgol):
       tif_list.append(os.path.join(savgol,i))
# Read metadata of first file
with rio.open(res_list[0]) as src0:
            meta = src0.meta
            transform_sg = src0.transform
# Update meta to reflect the number of layers
meta.update(count = len(tif_list)) 
# Read each layer and write it to stack
with rio.open(os.path.join(savgol,'savgol_stack.tif'), 'w', **meta) as dst:
        for id, layer in enumerate(tif_list, start=1):
            with rio.open(layer) as src1:
                dst.write_band(id, src1.read(1))

In [24]:
## Guardando el dataframe en un csv
df[["Date","month"]].to_csv(os.path.join(savgol,"data_SavGol.csv"),index=False)